In [1]:
import pandas as pd
import numpy as np
import collections

In [2]:
"""reviews = pd.read_csv('reviews_with_splits.csv')
reviews"""

"reviews = pd.read_csv('reviews_with_splits.csv')\nreviews"

In [4]:
"""by_rating = collections.defaultdict(list)
''' Iterating over the rows of the data frame using the iterrows method, to group the data by the rating then spliiting it inot train-test-val'''
for _, row in reviews.iterrows():
    by_rating[row.rating].append(row.to_dict())"""

The data set is 56000 training example, with a rating, review and split features. We need to group the rows by the rating using defaultdict method from the collections class. It outputs a list of dictinaries grouped by the rating.

In [3]:
def data_split_function(data, train_proportion, val_proportion, test_proportion, seed):
    final_list =[]
    np.random.seed(seed)
    for key, item_list in data.items():
        np.random.shuffle(item_list)
        n_total = len(data[key])
        n_train = int(train_proportion*n_total)
        n_val = int(val_proportion*n_total)
        n_test = int(test_proportion*n_total)
        for item in item_list[:n_train]:
            item['split'] = 'train'
        for item in item_list[n_train:n_train+n_val]:
            item['split'] = 'val'
        for item in item_list[n_val+n_train:n_val+n_test+n_test]:
             item['split'] = 'test'
        final_list.extend(item_list)
    return final_list

In [4]:
"""final_reviews = data_split_function(by_rating, 0.75, 0.15, 0.15, 42)
"""
final_reviews = pd.read_csv('final_reviews_csv')

In [8]:
 #This is a list of dictionaries and will be transformed to a pandas dataframe
#final_reviews = pd.DataFrame(final_reviews)
final_reviews = final_reviews.drop('Unnamed: 0', axis='columns')
final_reviews

,rating,review,split
0,negative,i signed online to schedule an appt . days pri...,train
1,negative,mct oil buy get the second half off . good dea...,train
2,negative,food here is mediocre and tastes super artific...,train
3,negative,server was not efficient and didn t seem to ha...,train
4,negative,service wasn t that great . the prime rib has ...,train
...,...,...,...
55995,positive,done a service repair for a broken sub . . fas...,test
55996,positive,not bad sushi . my husband and i went there ab...,train
55997,positive,we came here last night for dinner . we though...,train
55998,positive,all i have to say is start with the pineapple ...,train


In [8]:
#import re
"""def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([. , ! ?])", r"\1", text)
    text = re.sub(r"[^a-zA-Z., !?]+", r" ", text)
    return text
final_reviews.review = final_reviews.review.apply(preprocess_text)"""

In [11]:
#final_reviews.to_csv('final_reviews_csv')

In [9]:
from torch.utils.data import Dataset

In [10]:
class Vocabulary(object):
    def __init__(self, token_to_idx=None, add_unk=True, unk_token='<UNK>'):
        if token_to_idx is None:
            token_to_idx = {}
            self._token_to_idx = token_to_idx
        self._idx_to_token = {idx:token for token, idx in token_to_idx.items()}
        self._add_unk = add_unk
        self._unk_token = unk_token
        self.unk_index = -1
        if add_unk:
            self.unk_index = self.add_token(unk_token)
    def to_serializable(self):
        return {'token_to_idx': self._token_to_idx,
                'add_unk': self._add_unk,
                'unk_token': self._unk_token}
    @classmethod
    def from_serializable(cls, contents):
        return cls(**contents)
    def add_token(self, token):
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    def lookup_token(self, token):
        if self._add_unk:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]
    def lookup_index(self, index):
        if index not in self._idx_to_token:
            raise KeyError("The index (%d) is not in the Vocabulary" %index)
        return self._idx_to_token[index]
    def __str__(self):
        return "<Vocabulary(size = %d>" % len(self)
    def __len__(self):
        return len(self._token_to_idx)
    

In [11]:
class ReviewVectorizer(object):
    def __init__(self, review_vocab, rating_vocab):
        self.review_vocab = review_vocab
        self.rating_vocab = rating_vocab
    def vectorize(self, review):
        one_hot = np.zeros(len(self.review_vocab), dtype = np.float32)
        for token in review.split(" "):
            if token not in string.punctuation:
                one_hot[self.review_vocab.lookup_token(token)] = 1
        return one_hot
    @classmethod
    def from_dataframe(cls, review_df, cutoff = 25):
        review_vocab = Vocabulary(add_unk=True)
        rating_vocab = Vocabulary(add_unk= False)
        for rating in sorted(set(review_df.rating)):
            rating_vocab.add_token(rating)
        word_counts = Counter()
        for review in review_df.review:
            for word in review.split(" "):
                if word not in string.punctuation:
                    word_counts[word]+=1
        for word, count in word_counts.items():
            if count > cutoff:
                review_vocab.add_token(word)
        return cls(review_vocab, rating_vocab)
    @classmethod
    def from_serializable(cls, contents):
        review_vocab = Vocabulary.from_serializable(contents['review_vocab'])
        rating_vocab = Vocabulary.from_serializable(contents['rating_vocab'])
        return cls(review_vocab = review_vocab, rating_vocab = rating_vocab)
    def to_serializable(self):
        return {'review_vocab': self.review_vocab.to_serializable(),
                'rating_vocab': self.rating_vocab.to_serializable()}
        

In [12]:
def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device='cpu'):
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

In [20]:
"""import torch.nn as nn
import torch.nn.functional as F
class ReviewClassifier(nn.Module):
    def __init__(self, num_features):
        super(ReviewClassifier, self).__init__()
        self.fc1 = nn.Linear(in_features = num_features, out_features = 1)
    def forward(self, x_in, apply_sigmoid = False):
        y_out = self.fc1(x_in).squeeze()
        if apply_sigmoid:
            y_out = F.sigmoid(y_out)
        return y_out
        """

In [14]:
import torch.nn as nn
class ReviewMLP(nn.Module):
    def __inti__(self, input_dim, hidden_dim, output_dim):
        """ This is a multiLayer perceptron with two hidden layers 
        The input is a tensor with input_dim, the output of the first layer will be stored 
        in a hidden vector of the size hidden_dim * 1, and the output score vector will be of the size output_dim * 1 """
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(input_dim, hidden_dim)
    def forward(self, x_in, apply_softmax = False):
        layer1_output = self.fc1(x_in)
        hidden_vector = F.relu(layer1_output)
        prediction_vector = self.fc2(hidden_vector)
        if apply_softmax:
            prediction_vector = F.softmax(prediction_vector, dim=1)
        return prediction_vector
        

In [15]:
from argparse import Namespace
args = Namespace( model_state_file = 'model.pth', review_csv = 'final_reviews_csv', sav_dir = 'Home', vetorizer_file = 'vectorizer.json',
                 hidden_dim = 300,
                 batch_size = 128,
                 early_stopping_criteria = 5,
                 learning_rate = 0.001,
                 num_epochs = 100,
                 seed = 1337)


In [28]:
import torch.optim as optim
import torch
from torch.utils.data import DataLoader
from collections import Counter
import string
def make_train_state(args):
    return {'epoch_index':0, 
            'train_loss':[],
            'train_acc':[],
            'val_loss':[],
            'val_acc':[],
            'test_loss':-1,
            'test_acc':-1}
train_state = make_train_state(args)
if not torch.cuda.is_available():
    args.cuda = False
args.device = torch.device("cuda" if args.cuda else "cpu")

#dataset = ReviewDataset.load_dataset_and_make_vectorizer(args.review_csv)
vectorizer = dataset.get_vectorizer()
classifier = ReviewClassifier(num_features = len(vectorizer.review_vocab))
classifier = classifier.to(args.device)
loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr = args.learning_rate)
def compute_accuracy(y_predicted, y_target):
    correct = 0
    total = 0
    total+= y_target.size(0)
    correct+= (y_predicted == y_target).sum().item()
    accuracy = (correct/total) * 100
    return accuracy

In [29]:
for epoch_index in range(args.num_epochs):
    train_state['epoch_index'] = epoch_index
    dataset.set_split('train')
    batch_generator = generate_batches(dataset, args.batch_size)
    running_loss = 0.0
    running_acc = 0.0
    classifier.train()
    for batch_index, batch_dict in enumerate(batch_generator):
        optimizer.zero_grad()
        y_pred = classifier(x_in = batch_dict['x_data'].float())
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch - running_loss) / (batch_index+1)
        loss.backward()
        optimizer.step()
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index+1)
    train_state['train_loss'].append(running_loss)
    train_state['train_acc'].append(running_acc)
    dataset.set_split('val')
    batch_generator = generate_batches(dataset, batch_size = args.batch_size)
    running_loss = 0.
    running_acc = 0.
    classifier.eval()
    for batch_index, batch_dict in enumerate(batch_generator):
        y_pred = classifier(x_in = batch_dict['x_data'].float())
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch - running_loss) / (batch_index+1)
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc+= (acc_batch - running_acc) / (batch_index + 1)
    train_state['val_loss'].append(running_loss)
    train_state['val_acc'].append(running_acc)
    
        

In [33]:
dataset.set_split('test')
batch_generator = generate_batches(dataset, batch_size=args.batch_size)
running_loss = 0.0
running_acc = 0.0
for batch_index, batch_dict in enumerate(batch_generator):
    y_pred = classifier(x_in = batch_dict['x_data'].float())
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    batch_loss = loss.item()
    running_loss += (batch_loss- running_loss) / (batch_index+1)
    acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (batch_loss - running_acc) / (batch_index+1)
train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [35]:
print('Test loss: {:.3f}'.format(train_state['test_loss']))
print('Test accuracy: {:.2f}'.format(train_state['test_acc']))

Test loss: 0.324
Test accuracy: 0.32


In [39]:
def predict_rating(review, classifier, vectorizer, decision_threshold = 0.5):
    review = preprocess_text(review)
    vectorized_review = torch.tensor(vectorizer.vectorize(review))
    result = classifier(vectorized_review.view(1, -1))
    probability_value = F.sigmoid(result).item()
    index = 1
    if probability_value < decision_threshold:
        index = 0
    return vectorizer.rating_vocab.lookup_index(index)
test_review = 'I really had a good experience with this restaurant, but there are some negative sides to it, first: The atmosphere was hot and noisy, second : I did not  like the toast they offer, overall a decent resturant'
prediction = predict_rating(test_review, classifier, vectorizer)
print('{} -> {}'.format(test_review, prediction))
    

I really had a good experience with this restaurant, but there are some negative sides to it, first: The atmosphere was hot and noisy, second : I did not  like the toast they offer, overall a decent resturant -> negative


In [41]:
fc1_weights = classifier.fc1.weight.detach()[0]
_, indices = torch.sort(fc1_weights, dim=0, descending = True)
indices = indices.numpy().tolist()
print('Infuential words in positive reviews: ')

for i in range(20):
    print(vectorizer.review_vocab.lookup_index(indices[i]))

Infuential words in positive reviews: 
chinatown
pleasantly
mmmmmm
deliciousness
nclean
eclectic
hooked
amazed
artsy
spotless
heavenly
nexcellent
nhighly
stunning
keeper
chapel
mmmm
awesomeness
coma
nthank


In [42]:
print('Influntial words in negative review')
indices.reverse()
for i in range(20):
    print(vectorizer.review_vocab.lookup_index(indices[i]))

Influntial words in negative review
slowest
cancelled
unacceptable
nmaybe
underwhelmed
operator
meh
worst
subject
canceled
blech
insulting
gossiping
mediocre
burden
horrendous
awful
embarrassing
receipts
bland
